<a href="https://colab.research.google.com/github/emmcygn/DB_Tools/blob/master/GreenOre_Zambia_Pilot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GreenOre: Zambia Mining Pilot Project
## Comprehensive Technical Summary

### 1. Regional Analysis Capabilities

#### 1.1 Historical Dataset Overview
```python
print("USGS Dataset Characteristics:")
print(f"Total Records: 111")
print(f"Geographic Coverage: Zambia")
print(f"Primary Commodities: Copper, Emeralds")
print(f"Time Range: 1880-1960s")
```

#### 1.2 Regional Model Performance
```python
regional_metrics = {
    'Metric': ['Regional Model Accuracy', 'Producer Identification Precision',
               'Non-Producer Identification Precision'],
    'Score': [0.96, 1.00, 0.95]
}
```

#### 1.3 Regional Feature Importance
```python
regional_importance = {
    'Feature': ['Ore Type', 'Discovery Year', 'Host Rock Type',
                'Geographical Location', 'Other Factors'],
    'Impact (%)': [54.05, 20.09, 8.87, 17.00, 0.00]
}
```

### 2. Plot-Level Analysis System

#### 2.1 Data Collection Framework
- Comprehensive data collection form developed
- Key data points identified:
  * GPS coordinates and plot boundaries
  * Depth-yield relationships
  * Geological layering
  * Water table information
  * Production metrics

#### 2.2 Plot Optimization Model
```python
plot_features = {
    'Primary Features': ['Depth', 'Soil Type', 'Water Table', 'Yield'],
    'Secondary Features': ['Rock Type', 'Equipment', 'Processing Method'],
    'Geographic Features': ['GPS Coordinates', 'Plot Boundaries']
}
```

#### 2.3 Depth Optimization Capabilities
- Predictive modeling for optimal digging depths
- Confidence interval calculations
- Yield estimation based on depth and conditions
- Water table impact analysis

### 3. Visualization Capabilities

#### 3.1 Regional Visualization
- Interactive map with site classifications
- Heat map of mining activity
- Producer/processor facility distribution
- High-potential area identification

#### 3.2 Plot-Level Visualization
- Individual plot mapping
- Depth-yield relationship charts
- Geological layer visualization
- Success indicator mapping

### 4. Technical Achievements

#### 4.1 Data Integration
- USGS geological database processing
- Plot-level data structure development
- Multi-scale analysis capabilities
- Integrated visualization system

#### 4.2 Model Development
- Regional classification (96% accuracy)
- Plot-specific depth optimization
- Yield prediction framework
- Feature importance analysis

#### 4.3 Analysis Capabilities
- Regional trend identification
- Plot-specific recommendations
- Depth optimization
- Yield potential estimation

### 5. Business Applications

#### 5.1 Regional Level
- New site potential assessment
- Infrastructure planning support
- Risk evaluation framework
- Development priority mapping

#### 5.2 Individual Plot Level
- Optimal depth recommendations
- Yield potential estimates
- Resource optimization guidance
- Risk mitigation strategies

### 6. Development Roadmap

#### 6.1 Immediate Next Steps
1. Begin plot-level data collection
2. Implement depth optimization model
3. Deploy basic visualization tools
4. Start collecting real yield data

#### 6.2 Medium-term Goals
1. Integrate satellite imagery analysis
2. Develop user-friendly interface
3. Implement real-time data collection
4. Expand geological feature analysis

#### 6.3 Long-term Vision
1. Full-scale ASM optimization platform
2. Automated recommendation system
3. Integrated economic analysis
4. Environmental impact assessment

### 7. Current Limitations and Mitigation Strategies

#### 7.1 Data Limitations
- Limited historical data → Implementing new data collection
- Sparse yield records → Creating standardized reporting
- Inconsistent depth data → Developing measurement protocols

#### 7.2 Model Limitations
- Initial reliance on synthetic data → Gradual replacement with real data
- Limited feature set → Expanding through data collection
- Regional generalization → Plot-specific refinement

### 8. Success Metrics and KPIs

#### 8.1 Technical KPIs
- Model accuracy rates
- Prediction confidence levels
- Data collection completion rates
- System usage statistics

#### 8.2 Business KPIs
- Yield improvement rates
- Resource optimization metrics
- User adoption rates
- Economic impact indicators

**CODEBASE BELOW**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler

class ImprovedMiningClassifier:
    def __init__(self):
        self.label_encoders = {}
        self.model = None
        self.feature_importance = None
        self.scaler = StandardScaler()

    def prepare_features(self, df):
        """Prepare features with focus on most important ones"""
        df_copy = df.copy()
        features = pd.DataFrame(index=df_copy.index)

        # Process ore types (most important feature)
        df_copy['ore'] = df_copy['ore'].fillna('Unknown')
        self.label_encoders['ore'] = LabelEncoder()
        features['ore'] = self.label_encoders['ore'].fit_transform(df_copy['ore'])

        # Process discovery year
        mean_disc_yr = df_copy['disc_yr'].mean()
        features['disc_yr'] = df_copy['disc_yr'].fillna(mean_disc_yr)

        # Process host rock type
        df_copy['hrock_type'] = df_copy['hrock_type'].fillna('Unknown')
        self.label_encoders['hrock_type'] = LabelEncoder()
        features['hrock_type'] = self.label_encoders['hrock_type'].fit_transform(df_copy['hrock_type'])

        # Geographic features
        for coord in ['latitude', 'longitude']:
            features[coord] = df_copy[coord].fillna(df_copy[coord].mean())

        # Add engineered features
        # Combine nearby deposits information
        features['nearby_deposits'] = self.calculate_nearby_deposits(
            df_copy[['latitude', 'longitude']]
        )

        # Scale numerical features
        numerical_cols = ['disc_yr', 'latitude', 'longitude', 'nearby_deposits']
        features[numerical_cols] = self.scaler.fit_transform(features[numerical_cols])

        # Create target variable
        target = (df_copy['dev_stat'] == 'Producer').astype(int)

        return features, target

    def calculate_nearby_deposits(self, coordinates, radius=0.5):
        """Calculate number of deposits within radius degrees"""
        nearby_counts = []
        for idx, row in coordinates.iterrows():
            distances = np.sqrt(
                (coordinates['latitude'] - row['latitude'])**2 +
                (coordinates['longitude'] - row['longitude'])**2
            )
            nearby_counts.append(np.sum(distances < radius) - 1)  # Subtract self
        return np.array(nearby_counts)

    def train_model(self, features, target):
        """Train model with cross-validation and SMOTE"""
        X_train, X_test, y_train, y_test = train_test_split(
            features, target, test_size=0.2, random_state=42, stratify=target
        )

        # Apply SMOTE to balance training data
        smote = SMOTE(random_state=42)
        X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

        # Create and train model
        self.model = xgb.XGBClassifier(
            objective='binary:logistic',
            n_estimators=200,
            max_depth=4,
            learning_rate=0.05,
            scale_pos_weight=len(y_train[y_train==0]) / len(y_train[y_train==1]),
            eval_metric='logloss'
        )

        # Fit model
        self.model.fit(
            X_train_balanced,
            y_train_balanced,
            eval_set=[(X_test, y_test)],
            verbose=False
        )

        # Get feature importance
        importance_df = pd.DataFrame({
            'feature': features.columns,
            'importance': self.model.feature_importances_
        })
        self.feature_importance = importance_df.sort_values('importance', ascending=False)

        # Print model performance
        y_pred = self.model.predict(X_test)
        print("\nModel Performance:")
        print(classification_report(y_test, y_pred))

        return self.model, self.feature_importance

    def predict_potential(self, new_data):
        """Predict production potential with confidence scores"""
        if self.model is None:
            raise ValueError("Model needs to be trained first")

        prepared_data = self.prepare_features(new_data)[0]
        probabilities = self.model.predict_proba(prepared_data)

        return pd.DataFrame({
            'site_name': new_data['site_name'],
            'producer_probability': probabilities[:, 1],
            'confidence_score': np.max(probabilities, axis=1)
        })

def main():
    # Load data
    data_path = '/content/drive/MyDrive/Greenore_Zambia_Pilot/mrds-fZA-5zambia.csv'
    df = pd.read_csv(data_path)

    print("Data loaded successfully. Shape:", df.shape)

    # Initialize and train improved classifier
    classifier = ImprovedMiningClassifier()

    print("Preparing features...")
    features, target = classifier.prepare_features(df)
    print("Features prepared. Shape:", features.shape)

    print("Training model...")
    model, importance = classifier.train_model(features, target)

    print("\nFeature Importance:")
    print(importance)

    # Additional analysis
    producer_sites = df[df['dev_stat'] == 'Producer']
    print("\nNumber of producing sites:", len(producer_sites))
    print("\nProducing sites by ore type:")
    print(producer_sites['ore'].value_counts())

    return classifier, features, target

if __name__ == "__main__":
    classifier, features, target = main()

Data loaded successfully. Shape: (111, 46)
Preparing features...
Features prepared. Shape: (111, 6)
Training model...

Model Performance:
              precision    recall  f1-score   support

           0       0.95      1.00      0.98        20
           1       1.00      0.67      0.80         3

    accuracy                           0.96        23
   macro avg       0.98      0.83      0.89        23
weighted avg       0.96      0.96      0.95        23


Feature Importance:
           feature  importance
0              ore    0.771282
1          disc_yr    0.100345
3         latitude    0.049478
4        longitude    0.041153
5  nearby_deposits    0.029670
2       hrock_type    0.008072

Number of producing sites: 15

Producing sites by ore type:
ore
Anglesite, Descloizite, Galena, Hemimorphite, Pyromorphite, Smithsonite, Sphalerite, Willemite                1
Talc                                                                                                          1
Bornite,

**Map Visualiser:**

In [ ]:
import pandas as pd
import numpy as np
import folium
from folium import plugins
import requests
from datetime import datetime, timedelta
import json

class MiningYieldAnalyzer:
    def __init__(self):
        self.sites_df = None
        self.producers = None

    def load_site_data(self, csv_path):
        """Load and prepare mining site data"""
        self.sites_df = pd.read_csv(csv_path)
        # Filter for sites with known coordinates and production status
        self.sites_df = self.sites_df.dropna(subset=['latitude', 'longitude', 'dev_stat'])
        # Identify producers
        self.producers = self.sites_df[self.sites_df['dev_stat'] == 'Producer']
        return self.sites_df

    def create_yield_map(self):
        """Create an interactive map with yield estimates and clustering"""
        if self.sites_df is None:
            raise ValueError("Please load site data first")

        # Center map on Zambia
        center_lat = self.sites_df['latitude'].mean()
        center_lon = self.sites_df['longitude'].mean()

        # Create base map
        m = folium.Map(
            location=[center_lat, center_lon],
            zoom_start=7,
            tiles='OpenStreetMap'
        )

        # Add MarkerCluster for better visualization
        marker_cluster = plugins.MarkerCluster().add_to(m)

        # Add heatmap layer
        heat_data = [
            [row['latitude'], row['longitude'], 1.0 if row['dev_stat'] == 'Producer' else 0.5]
            for idx, row in self.sites_df.iterrows()
        ]
        plugins.HeatMap(heat_data).add_to(m)

        # Add individual markers with detailed information
        for idx, site in self.sites_df.iterrows():
            # Calculate potential score based on proximity to producers
            potential_score = self.calculate_site_potential(site)

            # Determine marker color based on status and potential
            if site['dev_stat'] == 'Producer':
                color = 'red'
            else:
                color = 'green' if potential_score > 0.7 else 'blue'

            # Create popup content
            popup_content = f"""
                <h4>{site['site_name']}</h4>
                <b>Status:</b> {site['dev_stat']}<br>
                <b>Ore:</b> {site.get('ore', 'Unknown')}<br>
                <b>Production Size:</b> {site.get('prod_size', 'Unknown')}<br>
                <b>Potential Score:</b> {potential_score:.2f}<br>
                <b>Nearby Producers:</b> {self.count_nearby_producers(site)}<br>
            """

            # Add marker
            folium.CircleMarker(
                location=[site['latitude'], site['longitude']],
                radius=8,
                color=color,
                fill=True,
                popup=folium.Popup(popup_content, max_width=300),
                tooltip=site['site_name']
            ).add_to(marker_cluster)

        # Add legend
        legend_html = '''
        <div style="position: fixed;
                    bottom: 50px; right: 50px; width: 150px; height: 90px;
                    border:2px solid grey; z-index:9999; background-color:white;
                    opacity:0.8;
                    font-size:12px;
                    padding: 10px">
          <h4>Site Status</h4>
          <div><span style="color: red;">●</span> Producer</div>
          <div><span style="color: green;">●</span> High Potential</div>
          <div><span style="color: blue;">●</span> Prospect</div>
        </div>
        '''
        m.get_root().html.add_child(folium.Element(legend_html))

        return m

    def calculate_site_potential(self, site):
        """Calculate site potential based on various factors"""
        if self.producers is None or len(self.producers) == 0:
            return 0.0

        # Calculate distances to all producers
        distances = []
        for _, producer in self.producers.iterrows():
            distance = np.sqrt(
                (site['latitude'] - producer['latitude'])**2 +
                (site['longitude'] - producer['longitude'])**2
            )
            distances.append(distance)

        # Calculate scores
        proximity_score = np.exp(-min(distances) * 10)  # Higher score for closer proximity
        cluster_score = np.exp(-np.mean(distances) * 5)  # Higher score for being in a cluster

        # Consider geological factors if available
        geology_score = 0.5  # Default score
        if 'ore' in site and site['ore'] in self.producers['ore'].values:
            geology_score = 1.0

        # Combine scores
        total_score = (0.4 * proximity_score +
                      0.3 * cluster_score +
                      0.3 * geology_score)

        return total_score

    def count_nearby_producers(self, site, threshold=0.5):
        """Count number of producers within threshold degrees"""
        if self.producers is None:
            return 0

        nearby = 0
        for _, producer in self.producers.iterrows():
            distance = np.sqrt(
                (site['latitude'] - producer['latitude'])**2 +
                (site['longitude'] - producer['longitude'])**2
            )
            if distance < threshold:
                nearby += 1

        return nearby

    def analyze_region(self, center_lat, center_lon, radius=0.5):
        """Analyze a specific region for mining potential"""
        if self.sites_df is None:
            raise ValueError("Please load site data first")

        # Find sites within radius
        region_sites = []
        for idx, site in self.sites_df.iterrows():
            distance = np.sqrt(
                (center_lat - site['latitude'])**2 +
                (center_lon - site['longitude'])**2
            )
            if distance < radius:
                region_sites.append({
                    'site_name': site['site_name'],
                    'distance': distance,
                    'status': site['dev_stat'],
                    'potential': self.calculate_site_potential(site)
                })

        return pd.DataFrame(region_sites)

def main():
    # Initialize analyzer
    analyzer = MiningYieldAnalyzer()

    # Load site data
    print("Loading site data...")
    sites_df = analyzer.load_site_data(
        '/content/drive/MyDrive/Greenore_Zambia_Pilot/mrds-fZA-5zambia.csv'
    )

    # Create visualization
    print("Creating interactive map...")
    yield_map = analyzer.create_yield_map()

    # Example analysis for a specific region
    print("\nAnalyzing example region...")
    example_region = analyzer.analyze_region(-13.133897, 27.849332, radius=0.5)
    print("\nRegion Analysis:")
    print(example_region)

    return analyzer, yield_map, example_region

if __name__ == "__main__":
    analyzer, yield_map, region_analysis = main()

Loading site data...
Creating interactive map...

Analyzing example region...

Region Analysis:
                           site_name  distance         status  potential
0                         Kabwe Mine  0.320348       Producer   0.550823
1                    Mindola - Nkana  0.461596       Producer   0.701347
2   (Facility) Nkana Copper Refinery  0.452068          Plant   0.514675
3                      Miku Prospect  0.472043     Occurrence   0.384114
4                     Chibuluma Mine  0.396883       Producer   0.701405
5    (Facility) Nkana Copper Smelter  0.452068          Plant   0.514675
6                       Kalushi East  0.369212       Prospect   0.385116
7                             Baluba  0.496264       Producer   0.701188
8         Nkana Rle Plant (Facility)  0.465545          Plant   0.464199
9                      Pitanda South  0.473540       Prospect   0.321123
10                           Pitanda  0.499809       Prospect   0.376328
11                          

In [ ]:
import folium
from folium import plugins
import pandas as pd

def create_detailed_map(df):
    """Create a detailed map with labels and analysis information"""

    # Center map on mean coordinates
    center_lat = df['latitude'].mean()
    center_lon = df['longitude'].mean()

    # Create base map
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=9,
        tiles='OpenStreetMap'
    )

    # Add scale bar
    folium.plugins.MousePosition().add_to(m)
    folium.plugins.MeasureControl().add_to(m)

    # Create a feature group for the heatmap
    heat_data = [[row['latitude'], row['longitude']] for _, row in df.iterrows()]
    plugins.HeatMap(heat_data, radius=15).add_to(m)

    # Create a feature group for site markers
    for idx, site in df.iterrows():
        # Determine marker color based on status
        if 'Producer' in str(site['dev_stat']):
            color = 'red'
            radius = 12
        elif 'Plant' in str(site['dev_stat']):
            color = 'orange'
            radius = 10
        else:
            color = 'blue'
            radius = 8

        # Create popup content
        popup_content = f"""
            <div style="font-family: Arial; width: 200px;">
                <h4>{site['site_name']}</h4>
                <b>Status:</b> {site['dev_stat']}<br>
                <b>Potential:</b> {site['potential']:.2f}<br>
                <b>Distance:</b> {site['distance']:.2f}°<br>
            </div>
        """

        # Add permanent label for high potential sites
        if site['potential'] > 0.5:
            folium.Rectangle(
                bounds=[[site['latitude']-0.01, site['longitude']-0.01],
                       [site['latitude']+0.01, site['longitude']+0.01]],
                color='green',
                fill=True,
                popup=f"High Potential Area: {site['potential']:.2f}",
                weight=1
            ).add_to(m)

        # Add circle marker
        folium.CircleMarker(
            location=[site['latitude'], site['longitude']],
            radius=radius,
            color=color,
            fill=True,
            popup=folium.Popup(popup_content, max_width=300),
            tooltip=f"{site['site_name']} ({site['potential']:.2f})"
        ).add_to(m)

        # Add site name label for important sites
        if site['potential'] > 0.4:
            folium.Popup(
                site['site_name'],
                permanent=True
            ).add_to(folium.CircleMarker(
                location=[site['latitude'], site['longitude']],
                radius=1,
                color='none',
                fill=False
            ).add_to(m))

    # Add legend
    legend_html = '''
    <div style="position: fixed;
                bottom: 50px; right: 50px; width: 200px;
                border:2px solid grey; z-index:9999; background-color:white;
                opacity:0.8; padding: 10px; font-size:12px;">
        <h4>Site Classification</h4>
        <div style="display: flex; align-items: center; margin-bottom: 5px;">
            <span style="background-color: red; width: 12px; height: 12px;
                   display: inline-block; margin-right: 5px; border-radius: 50%;"></span>
            Producer
        </div>
        <div style="display: flex; align-items: center; margin-bottom: 5px;">
            <span style="background-color: orange; width: 12px; height: 12px;
                   display: inline-block; margin-right: 5px; border-radius: 50%;"></span>
            Processing Plant
        </div>
        <div style="display: flex; align-items: center; margin-bottom: 5px;">
            <span style="background-color: blue; width: 12px; height: 12px;
                   display: inline-block; margin-right: 5px; border-radius: 50%;"></span>
            Prospect
        </div>
        <div style="display: flex; align-items: center;">
            <span style="background-color: #00FF00; width: 12px; height: 12px;
                   display: inline-block; margin-right: 5px;"></span>
            High Potential Area
        </div>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))

    return m

def main():
    df = pd.read_csv('/content/drive/MyDrive/Greenore_Zambia_Pilot/mrds-fZA-5zambia.csv')

    #example potential scores
    example_potentials = {
        'Kabwe Mine': 0.550823,
        'Mindola': 0.461596,
        'Baluba': 0.701188,
        'Nkana Copper Refinery': 0.514675
        # Add more as needed
    }

    # Add potential scores to dataframe
    df['potential'] = df['site_name'].map(example_potentials).fillna(0.3)


    df['distance'] = 0.5

    # Create and display the map
    mining_map = create_detailed_map(df)
    return mining_map

if __name__ == "__main__":
    mining_map = main()
    display(mining_map)

Mining Depth Optimisation Model Draft:

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

class MiningDepthOptimizer:
    def __init__(self):
        self.model = None
        self.label_encoders = {}
        self.scaler = StandardScaler()
        self.feature_importance = None

    def prepare_data(self, data):
        """Prepare mining data for modeling"""
        df = pd.DataFrame(data)

        # Handle categorical variables
        categorical_cols = ['soil_type', 'rock_type']
        for col in categorical_cols:
            if col in df.columns:
                self.label_encoders[col] = LabelEncoder()
                df[col] = self.label_encoders[col].fit_transform(df[col])

        # Scale numerical features
        numerical_cols = ['water_table', 'latitude', 'longitude']
        if any(col in df.columns for col in numerical_cols):
            df_nums = df[numerical_cols]
            df[numerical_cols] = self.scaler.fit_transform(df_nums)

        return df

    def train_model(self, X, y):
        """Train the depth optimization model"""
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )

        # Initialize and train model
        self.model = xgb.XGBRegressor(
            objective='reg:squarederror',
            n_estimators=100,
            max_depth=4,
            learning_rate=0.1
        )

        # Train model
        self.model.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            verbose=False
        )

        # Calculate feature importance
        self.feature_importance = pd.DataFrame({
            'feature': X.columns,
            'importance': self.model.feature_importances_
        }).sort_values('importance', ascending=False)

        # Evaluate model
        y_pred = self.model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        return {
            'mse': mse,
            'r2': r2,
            'feature_importance': self.feature_importance
        }

    def predict_optimal_depth(self, site_features):
        """Predict optimal mining depth for new sites"""
        if self.model is None:
            raise ValueError("Model needs to be trained first")

        # Prepare features
        site_df = pd.DataFrame([site_features])
        prepared_features = self.prepare_data(site_df)

        # Make prediction
        predicted_depth = self.model.predict(prepared_features)[0]

        # Get confidence intervals (using model variance as proxy)
        predictions = []
        for _ in range(100):
            pred = self.model.predict(prepared_features)
            predictions.append(pred[0])

        confidence_interval = np.percentile(predictions, [5, 95])

        return {
            'optimal_depth': predicted_depth,
            'confidence_interval': confidence_interval,
            'confidence_range': confidence_interval[1] - confidence_interval[0]
        }

    def analyze_depth_yield_relationship(self, data):
        """Analyze relationship between depth and yield"""
        df = pd.DataFrame(data)

        plt.figure(figsize=(10, 6))
        plt.scatter(df['depth'], df['yield'])
        plt.xlabel('Depth (meters)')
        plt.ylabel('Yield (kg)')
        plt.title('Depth vs Yield Relationship')

        # Add trend line
        z = np.polyfit(df['depth'], df['yield'], 1)
        p = np.poly1d(z)
        plt.plot(df['depth'], p(df['depth']), "r--", alpha=0.8)

        return plt

# Example usage with synthetic data
def generate_synthetic_data(n_samples=100):
    """Generate synthetic mining data for testing"""
    np.random.seed(42)

    # Generate basic features
    data = []
    for _ in range(n_samples):
        depth = np.random.uniform(1, 50)  # Depths between 1-50m

        # Create yield with some realistic patterns
        base_yield = depth * 10  # Base yield increases with depth
        noise = np.random.normal(0, base_yield * 0.2)  # Add some noise
        water_table = np.random.uniform(20, 40)  # Water table depth

        # Reduce yield if below water table
        if depth > water_table:
            base_yield *= 0.7

        # Add some randomization for soil types
        soil_types = ['Clay', 'Sandy', 'Rocky', 'Mixed']
        rock_types = ['Soft', 'Medium', 'Hard']

        site_data = {
            'depth': depth,
            'yield': max(0, base_yield + noise),
            'water_table': water_table,
            'soil_type': np.random.choice(soil_types),
            'rock_type': np.random.choice(rock_types),
            'latitude': np.random.uniform(-13.5, -13.0),
            'longitude': np.random.uniform(27.5, 28.0)
        }
        data.append(site_data)

    return data

def main():
    # Generate synthetic data
    print("Generating synthetic mining data...")
    mining_data = generate_synthetic_data(200)

    # Initialize and train model
    optimizer = MiningDepthOptimizer()

    # Prepare features and target
    df = pd.DataFrame(mining_data)
    X = df.drop('yield', axis=1)
    y = df['yield']

    # Train model
    print("\nTraining model...")
    results = optimizer.train_model(X, y)

    print("\nModel Performance:")
    print(f"Mean Squared Error: {results['mse']:.2f}")
    print(f"R² Score: {results['r2']:.2f}")

    print("\nFeature Importance:")
    print(results['feature_importance'])

    # Example prediction
    new_site = {
        'depth': 25,
        'water_table': 30,
        'soil_type': 'Rocky',
        'rock_type': 'Medium',
        'latitude': -13.2,
        'longitude': 27.8
    }

    prediction = optimizer.predict_optimal_depth(new_site)
    print("\nExample Prediction:")
    print(f"Optimal Depth: {prediction['optimal_depth']:.2f} meters")
    print(f"Confidence Interval: {prediction['confidence_interval'][0]:.2f} - {prediction['confidence_interval'][1]:.2f} meters")

    return optimizer, mining_data

if __name__ == "__main__":
    optimizer, data = main()

Generating synthetic mining data...

Training model...


ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:soil_type: object, rock_type: object

Collection Form:

# Mining Site Data Collection Form

## 1. Basic Information
- Date: [DD/MM/YYYY]
- Miner ID: _____________
- Plot License Number: _____________

## 2. Location Details
- Plot Corner Coordinates (GPS):
  * Northeast: [Latitude] _______ [Longitude] _______
  * Northwest: [Latitude] _______ [Longitude] _______
  * Southeast: [Latitude] _______ [Longitude] _______
  * Southwest: [Latitude] _______ [Longitude] _______
- Total Plot Size (hectares): _______
- Nearest Town/Village: _____________
- Distance to Nearest Processing Facility (km): _______

## 3. Current Mining Activities
### 3.1 Active Digging Sites
For each active digging site within your plot:

Site 1:
- GPS Location: [Latitude] _______ [Longitude] _______
- Current Depth (meters): _______
- Width of Excavation (meters): _______
- Length of Excavation (meters): _______
- Started Mining Date: [MM/YYYY]

Site 2: [Same format as above...]

### 3.2 Production Data (Last 30 Days)
- Total Ore Extracted (kg): _______
- Estimated Grade (%): _______
- Processing Method Used: [Circle]
  * Hand Sorting
  * Washing
  * Crushing
  * Other: _____________

## 4. Geological Information
### 4.1 Surface Layer
- Main Soil Type: [Circle]
  * Clay
  * Sandy
  * Rocky
  * Mixed
  * Other: _____________
- Color of Surface Soil: _____________

### 4.2 Underground Layers
Starting from top, describe each distinct layer:

Layer 1:
- Depth Range (meters): _______ to _______
- Material Type: _____________
- Color: _____________
- Hardness [Circle]: Soft / Medium / Hard
- Mineral Indicators Present: _____________

Layer 2: [Same format as above...]

### 4.3 Water Table
- Depth to Water Table (if encountered, meters): _______
- Water Management Issues: [Yes/No]
- If Yes, Describe: _____________

## 5. Historical Information
### 5.1 Previous Mining Areas
For each previously mined area:

Area 1:
- GPS Location: [Latitude] _______ [Longitude] _______
- Maximum Depth Reached (meters): _______
- Total Ore Extracted (estimated kg): _______
- Period Mined: [MM/YYYY] to [MM/YYYY]
- Reason Stopped: _____________

Area 2: [Same format as above...]

## 6. Success Indicators
### 6.1 Best Performing Site
- GPS Location: [Latitude] _______ [Longitude] _______
- Depth Range of Best Yields (meters): _______ to _______
- Best Daily Yield (kg): _______
- Month/Year of Best Yield: [MM/YYYY]
- Description of Mineral Indicators: _____________

### 6.2 Challenges
- Main Challenges Encountered: [Circle all that apply]
  * Water Management
  * Hard Rock
  * Equipment Limitations
  * Processing Issues
  * Transportation
  * Other: _____________

## 7. Additional Information
### 7.1 Equipment Used
- List Main Equipment: _____________
- Maximum Digging Depth Capability (meters): _______

### 7.2 Photos Required
Please attach photos of:
1. Overall plot view
2. Active mining sites
3. Rock/soil samples from productive areas
4. Any visible mineral indicators
5. Different soil/rock layers in excavations

### 7.3 Additional Notes
Any other observations or information you think might be helpful:
_____________________________________________
_____________________________________________

---
For Office Use Only:
Form ID: _____________
Data Entry Date: _____________
Verified By: _____________

Model V2 Using Satellite Sources

In [ ]:
!pip install geemap
!pip install earthengine-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.0 MB/s eta 0:00:00


In [ ]:
import ee
ee.Authenticate()

In [ ]:
import folium
from folium import plugins
import pandas as pd
import requests
from datetime import datetime, timedelta

class SimpleSatelliteMiningMap:
    def __init__(self):
        self.satellite_layers = {
            'ESRI Satellite': 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            'OpenTopoMap': 'https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png',
            'Stamen Terrain': 'https://stamen-tiles-{s}.a.ssl.fastly.net/terrain/{z}/{x}/{y}.jpg'
        }

    def create_base_map(self, center_lat, center_lon, zoom_start=7):
        """Create base map with multiple layer options"""
        # Create the base map with OpenStreetMap
        m = folium.Map(
            location=[center_lat, center_lon],
            zoom_start=zoom_start,
            tiles='OpenStreetMap',
            name='OpenStreetMap'
        )

        # Add satellite and terrain layers
        for name, url in self.satellite_layers.items():
            folium.TileLayer(
                tiles=url,
                attr=f'Tiles courtesy of {name}',
                name=name,
                overlay=False
            ).add_to(m)

        return m

    def add_mining_data(self, m, mining_df):
        """Add mining site data with clustering"""
        # Create feature groups for different site types
        producers = folium.FeatureGroup(name='Producers')
        prospects = folium.FeatureGroup(name='Prospects')
        facilities = folium.FeatureGroup(name='Processing Facilities')

        # Create heatmap data
        heat_data = []

        for idx, row in mining_df.iterrows():
            # Determine site type and style
            if row['dev_stat'] == 'Producer':
                color = 'red'
                group = producers
                weight = 1.0
            elif 'Plant' in str(row['dev_stat']):
                color = 'orange'
                group = facilities
                weight = 0.8
            else:
                color = 'blue'
                group = prospects
                weight = 0.5

            # Add to heatmap data
            heat_data.append([
                row['latitude'],
                row['longitude'],
                weight
            ])

            # Create popup content
            popup_content = f"""
                <div style="width:200px">
                    <h4>{row['site_name']}</h4>
                    <b>Status:</b> {row['dev_stat']}<br>
                    <b>Type:</b> {row.get('ore', 'Unknown')}<br>
                    <b>Production:</b> {row.get('prod_size', 'Unknown')}<br>
                </div>
            """

            # Add marker to appropriate group
            folium.CircleMarker(
                location=[row['latitude'], row['longitude']],
                radius=8,
                color=color,
                fill=True,
                popup=folium.Popup(popup_content, max_width=300),
                tooltip=row['site_name']
            ).add_to(group)

        # Add all groups to map
        producers.add_to(m)
        prospects.add_to(m)
        facilities.add_to(m)

        # Add heatmap layer
        plugins.HeatMap(
            heat_data,
            name='Heat Map',
            min_opacity=0.3,
            max_zoom=18,
            radius=25,
            blur=15,
            overlay=True
        ).add_to(m)

        return m

    def add_map_features(self, m):
        """Add additional map features and controls"""
        # Add layer control
        folium.LayerControl().add_to(m)

        # Add fullscreen option
        plugins.Fullscreen().add_to(m)

        # Add coordinate display
        plugins.MousePosition().add_to(m)

        # Add measurement tools
        plugins.MeasureControl(
            position='bottomleft',
            primary_length_unit='kilometers',
            secondary_length_unit='miles',
            primary_area_unit='sqkilometers',
            secondary_area_unit='acres'
        ).add_to(m)

        # Add mini map
        mini_map = plugins.MiniMap(toggle_display=True)
        m.add_child(mini_map)

        return m

    def add_custom_legend(self, m):
        """Add custom legend to map"""
        legend_html = '''
        <div style="position: fixed;
                    bottom: 50px; right: 50px; width: 180px;
                    border:2px solid grey; z-index:9999;
                    background-color:white;
                    opacity:0.8;
                    padding: 10px;
                    font-size:12px;">
        <p style="margin-bottom:10px"><strong>Site Classification</strong></p>
        <div>
            <span style="background-color: red;
                        width: 15px;
                        height: 15px;
                        display: inline-block;
                        margin-right: 5px;
                        border-radius: 50%;"></span>
            Producer Sites
        </div>
        <div>
            <span style="background-color: orange;
                        width: 15px;
                        height: 15px;
                        display: inline-block;
                        margin-right: 5px;
                        border-radius: 50%;"></span>
            Processing Facilities
        </div>
        <div>
            <span style="background-color: blue;
                        width: 15px;
                        height: 15px;
                        display: inline-block;
                        margin-right: 5px;
                        border-radius: 50%;"></span>
            Prospects
        </div>
        <div style="margin-top:5px;">
            <span style="background: linear-gradient(to right, blue, red);
                        width: 50px;
                        height: 15px;
                        display: inline-block;
                        margin-right: 5px;"></span>
            Activity Heatmap
        </div>
        </div>
        '''
        m.get_root().html.add_child(folium.Element(legend_html))

        return m

    def create_map(self, mining_df):
        """Create complete map with all features"""
        # Calculate map center
        center_lat = mining_df['latitude'].mean()
        center_lon = mining_df['longitude'].mean()

        # Create base map
        m = self.create_base_map(center_lat, center_lon)

        # Add mining data
        m = self.add_mining_data(m, mining_df)

        # Add features
        m = self.add_map_features(m)

        # Add legend
        m = self.add_custom_legend(m)

        return m

def main():
    # Load mining data
    mining_df = pd.read_csv('/content/drive/MyDrive/Greenore_Zambia_Pilot/mrds-fZA-5zambia.csv')

    # Create map
    mapper = SimpleSatelliteMiningMap()
    mining_map = mapper.create_map(mining_df)

    return mining_map

if __name__ == "__main__":
    mining_map = main()
    display(mining_map)